In [2]:
import pandas as pd
import numpy as np
import urllib
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


In [3]:
df = pd.read_table('toxicity_annotated_comments.tsv')
df2 = pd.read_table('toxicity_annotations.tsv')

In [4]:
df2.head()

,rev_id,worker_id,toxicity,toxicity_score
0,2232.0,723,0,0.0
1,2232.0,4000,0,0.0
2,2232.0,3989,0,1.0
3,2232.0,3341,0,0.0
4,2232.0,1574,0,1.0


In [5]:
df2.head()

,rev_id,worker_id,toxicity,toxicity_score
0,2232.0,723,0,0.0
1,2232.0,4000,0,0.0
2,2232.0,3989,0,1.0
3,2232.0,3341,0,0.0
4,2232.0,1574,0,1.0


In [ ]:

from nltk.tokenize import sent_tokenize, word_tokenize

w = word_tokenize("gaming, the gamers play games")
w

In [11]:
import string
from nltk.corpus import stopwords

sample = 'sample message! Cool, punctuation'

def Depunc(text):
    text = text.replace('NEWLINE_TOKEN','\n')
    nopunc = [char if char not in string.punctuation 
              else ' ' for char in text]
    nopunc = ''.join(nopunc)
    return nopunc

def Clean(text):
    clean = [word for word in text.split()
             if word.lower() not in stopwords.words('english')]
    return clean

from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
porter_stemmer.stem('words')
from nltk.tokenize import sent_tokenize, word_tokenize

df['cleaned'] = df['comment'].apply(lambda x: Depunc(x))
df['cleaned'] = df['cleaned'].apply(lambda x: word_tokenize(x))

df['list'] = df['cleaned'].apply(lambda x: Clean(x))
df.head()



LookupError: 
**********************************************************************
  Resource 'tokenizers/punkt/english.pickle' not found.  Please
  use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - 'C:\\Users\\david/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\david\\Anaconda3\\nltk_data'
    - 'C:\\Users\\david\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\david\\AppData\\Roaming\\nltk_data'
    - ''
**********************************************************************

In [10]:


def stem(text):
    for i in text:
        i = porter_stemmer.stem(i)
    return text
    
    #return [porter_stemmer.stem(x) for x in text]

df['list'] = df['list'].apply(lambda x: stem(x))

RecursionError: maximum recursion depth exceeded in comparison

In [ ]:
df = df.join(df2, lsuffix='rev_id', rsuffix='rev_id')

In [ ]:
df[df.toxicity == 1].list[3480]

In [ ]:
words = []
def add_words(text):
    [words.append(i) for i in text if i not in words]
    
df['list'].apply(lambda x: add_words(x))
len(words)

In [ ]:
wordscores = pd.DataFrame({'score':0, 'word':words, 'count':0})
wordscores.head()


In [ ]:
def scores(text):
    score = 1.0 / len(text)
    
    for i in text:
        wordscores.loc[wordscores.index[wordscores['word']
                        == i].tolist()[0], 'score'] += score
    
    #[wordscores.loc[wordscores.index[wordscores['word']
    #                == i].tolist()[0],'count'] += score for i in text]
    
def inc_count(text):
    
    for i in text:
        wordscores.loc[wordscores.index[wordscores['word']
                        == i].tolist()[0], 'count'] += 1
    
    #[wordscores.loc[wordscores.index[wordscores['word']
    #                == i].tolist()[0],'count'] += 1 for i in text]
    
#wordscores.loc[wordscores.index[wordscores['word'] == i].tolist()[0],'count'] = 2

    

df[df.toxicity == 1].list.apply(lambda x: scores(x))
df.list.apply(lambda x: inc_count(x))

In [ ]:
wordscores.loc[wordscores.index[wordscores['word'] == 'like'], 'count'] += 1


In [ ]:
#wordscores['count'] = wordscores['count'].apply(lambda x: x + 1)
wordscores['toxicity'] = wordscores['score'] / wordscores['count']


In [ ]:
wordscores.sort_values('toxicity', ascending=False, inplace=True)

In [ ]:
wordscores = wordscores[wordscores['count'] > 1]

In [ ]:
wordscores